In [ ]:
import requests
import pandas as pd
import numpy as np
import re
import time
from google.colab import files # импорт/экспорт файлов

In [ ]:
# цель - анализ финансовых показателей компаний среди разработчиков информационных технологий

# Создание списка компаний

In [ ]:
url1 = 'https://www.testfirm.ru/'

## 1 вариант - Список из первых 50 фирм - с первой страницы фильтра

In [ ]:
url_list = 'https://www.testfirm.ru/rating' # рейтинг по всем предприятиям 
k_1 = 63   #код деятельности
k_2 = "sverdlovskaya-oblast"  #область
url_list_copm = url_list + '/' + str(k_1) + '/' + k_2 + '/'
text = requests.get(url_list_copm) # 
try:
  text.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [ ]:
# поиск названия компании
text_write = text.text

def find_komp(text_write, N):
    """ф. для поиска ИНН и названия по каждой компании - для формирования url-ссылки на информацию по этой компании
    """
    firma_list = text_write.split('org-link" href="/result/')
    return firma_list[N].split('"')[0]

def find_komp_re(text_write, N):
  """  ф. для поиска ИНН и названия по каждой компании - с помощью регулярных выражений
  """
  pass    

## 2 вариант - Список из всех фирм фильтра

### - загрузим страницу с рейтингом компаний

In [ ]:
# Рейтинг организаций по выручке - Вид деятельности: 63 "Деятельность в области информационных технологий"
url_list_all = url1 + 'rating/63/' # адрес страницы с рейтингом
text_list_all = requests.get(url_list_all) # 
try:
  text_list_all.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [ ]:
def find_name(url):
    """ ф. находит инфу для url страницы с информацией по компании - состоит из ИНН и названия Компании
    -  inn_firma 
    """
    text = requests.get(url)
    text_find = text.text 
    text_find = text_find.split('class="org-link"')[1].split('result/')[1].split('">')[0]
    return text_find

def find_name_next(url, koef = 1):
    """ ф. находит инфу для url страницы с информацией по следующей компании
    """
    text = requests.get(url)
    text_find = text.text 
    text_find = text_find.split('class="org-link"')[koef].split('result/')[1].split('">')[0]
    return text_find  

In [ ]:
#url_one = 'https://www.testfirm.ru/result/' + name #   https://www.testfirm.ru/result/7708119944_ooo-sibirskaya-internet-kompaniya

def find_info_comp(url_one):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку
    """
    stroka = [] # собираем информацию сюда

    text_one = requests.get(url_one)
    text_write = text_one.text
    a1 = text_write.split('Организация: ')[1]
 #   stroka.append(a1.replace(' &quot;', '').split('<')[0].split(' ')[0]) # правовая форма
    if text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0]: # название
        stroka.append(text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0])
    else:
        stroka.append(0)
#    print(stroka[-1])
    stroka.append(a1.split('ИНН: ')[1][:10]) # ИНН
#    print(stroka[-1])
    stroka.append(a1.split('<b>')[2].split('<')[0].rstrip()) # активы
#    print(stroka[-1])
    mln = a1.split('Активы на ')[1].split('/b>')[1][:5].rstrip() # млн или тыс
    if mln in 'млн.':
        stroka.append(1000000)
    elif mln in 'тыс.':
        stroka.append(1000)
#    print(stroka[-1])
    stroka.append(a1.split('Выручка')[1].split('b>')[1].split('<')[0].rstrip()) # выручка
#    print(stroka[-1])
    mln = a1.split('Выручка')[1].split('b>')[2].split('руб')[0].rstrip() # млн ии тыс
    if mln in 'млн.':
        stroka.append(1000000)
    elif mln in 'тыс.':
        stroka.append(1000)
#    print(stroka[-1])
    if 'численности ' in text_write:
        stroka.append(a1.split('численности ')[1].split('b>')[1].split('<')[0]) # численность человек
#        print(stroka[-1])
    else:
        stroka.append(0) 
    if 'Дополнительные данные ФНС' in text_write:
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[1].split('<')[0].rstrip()) # доходы 
#        print(stroka[-1])
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[2].split('<')[0].rstrip()) # расходы
#        print(stroka[-1])
    else:
        stroka.append(0)
        stroka.append(0)
    if "Уплачено налогов" in text_write:
        stroka.append(text_write.split("Уплачено налогов")[1].split('b>')[1].split('<')[0].rstrip()) # налоги уплаченные
    else:
        stroka.append(0)
    return stroka

# другая функция - по-быстрее
def find_info_comp_primer(url_one):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку
    """
    stroka = [] # собираем информацию сюда

    text_one = requests.get(url_one)
    text_write = text_one.text
    a1 = text_write.split('id="searchText" name="s" value=')[1]
    a2 = a1.split(' placeholder')[0] # name
    stroka.append(a2)

    a2 = a1.split('ИНН: ')[1].split(' ')[0] # INN
    stroka.append(a2)

    a2 = a1.split('Активы на ')[1].split('<b>')[1].split('</b>') # aktiv
    a2[0] # aktiv
    mln = a2[1].split(' (')[0] # млн.
    if 'млн.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(1)
    elif 'тыс.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(0.001)

    a2 = a1.split('Выручка за ')[1].split('<b>')[1].split(' </b>') # выручка
    mln = a2[1].split(' (')[0] # млн.
    if 'млн.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(1)
    elif 'тыс.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(0.001)

    if 'численности ' in text_write:
        stroka.append(a1.split('численности ')[1].split('b>')[1].split('<')[0]) # численность человек
    else:
        stroka.append(0) 

    if 'Дополнительные данные ФНС' in text_write:
        a2 = a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[1].split('<')[0].rstrip().replace(' ', '')
        stroka.append(a2) # доходы 
        a2 = a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[2].split('<')[0].rstrip().replace(' ', '')
        stroka.append(a2) # расходы
    else:
        stroka.append(0)
        stroka.append(0)

    if "Уплачено налогов" in text_write:
        stroka.append(text_write.split("Уплачено налогов")[1].split('b>')[1].split('<')[0].rstrip()) # налоги уплаченные
    else:
        stroka.append(0)
    
    return stroka

In [ ]:
#
t0 = time.time()
Table_find_values = [] 

name = find_name(url_list_all)      # ф. найдет определитель компании - inn_firma
url_one = url1 + 'result/' + name   # url страницы с информацией о компании
text_one = requests.get(url_one)    # создат объект Response
a = find_info_comp_primer(url_one)             # ф. покажет инф. о компании
Table_find_values.append(a)
inn = name.split('_')[0]            # выделить инн
url_1 = url1 + 'rating/63_11_1/' + inn + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм
print('\t \t 1')

name_next_1 = requests.get(url_1).text.split(name)[1].split('href="/result/')[1].split('">')[0] # найти name следующей компании
inn_next = name_next_1.split('_')[0]
url_next_company = url1 + 'result/' + name_next_1
a = find_info_comp_primer(url_next_company) #  ф. покажет инф. о компании
Table_find_values.append(a)
url_1 = url1 + 'rating/63_11_1/' + inn_next + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм
print('\t \t 2')

for i in range(558):
    name_next_1 = requests.get(url_1).text.split(name_next_1)[1].split('href="/result/')[1].split('">')[0] # найти name следующей компании
    inn_next = name_next_1.split('_')[0]
    url_next_company = url1 + 'result/' + name_next_1
    a = find_info_comp_primer(url_next_company) #  ф. покажет инф. о компании
    Table_find_values.append(a)
    url_1 = url1 + 'rating/63_11_1/' + inn_next + '/' # url страницы с местом в рейтинге  / вернутся в рейтинг фирм 
    print('\t \t', i + 3)
t1 = time.time() - t0
print(t1)
# возможно ли оптимизировать время, если подправить функцию сбора данных по странице

	 	 1
	 	 2
	 	 3
	 	 4
	 	 5
	 	 6
	 	 7
	 	 8
	 	 9
	 	 10
	 	 11
	 	 12
	 	 13
	 	 14
	 	 15
	 	 16
	 	 17
	 	 18
	 	 19
	 	 20
	 	 21
	 	 22
	 	 23
	 	 24
	 	 25
	 	 26
	 	 27
	 	 28
	 	 29
	 	 30
	 	 31
	 	 32
	 	 33
	 	 34
	 	 35
	 	 36
	 	 37
	 	 38
	 	 39
	 	 40
	 	 41
	 	 42
	 	 43
	 	 44
	 	 45
	 	 46
	 	 47
	 	 48
	 	 49
	 	 50
	 	 51
	 	 52
	 	 53
	 	 54
	 	 55
	 	 56
	 	 57
	 	 58
	 	 59
	 	 60
	 	 61
	 	 62
	 	 63
	 	 64
	 	 65
	 	 66
	 	 67
	 	 68
	 	 69
	 	 70
	 	 71
	 	 72
	 	 73
	 	 74
	 	 75
	 	 76
	 	 77
	 	 78
	 	 79
	 	 80
	 	 81
	 	 82
	 	 83
	 	 84
	 	 85
	 	 86
	 	 87
	 	 88
	 	 89
	 	 90
	 	 91
	 	 92
	 	 93
	 	 94
	 	 95
	 	 96
	 	 97
	 	 98
	 	 99
	 	 100
	 	 101
	 	 102
	 	 103
	 	 104
	 	 105
	 	 106
	 	 107
	 	 108
	 	 109
	 	 110
	 	 111
	 	 112
	 	 113
	 	 114
	 	 115
	 	 116
	 	 117
	 	 118
	 	 119
	 	 120
	 	 121
	 	 122
	 	 123
	 	 124
	 	 125
	 	 126
	 	 127
	 	 128
	 	 129
	 	 130
	 	 131
	 	 132
	 	 133
	 	 134
	 	 135
	 	 136
	 	 137
	 	 138
	 	 

In [ ]:
print('время сбора данных для 6500 компаний- \t{} мин.'.format(round( t1 / 60, 2))) # 217.5 мин./ 177.87 мин. после корректировки функции
# 183.09 мин.
# для 560 комп - 15.62 мин.

время сбора данных для 6500 компаний- 	15.62 мин.


In [ ]:
177.87 / 60 # сколько часов для сбора данных для 6500 позиций

2.9645

# Сбор данных по каждой компании - тут только тестирование

In [ ]:
# надо бы дописать эту функцию
def find_info_comp_re(List_all_komp, n_comp):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку с помощью регулярных выражений
    """
    pass

In [ ]:
# уменьшение скорости выполнения сбора данных со страницы
#url_one = 'https://www.testfirm.ru/result/' + name #   https://www.testfirm.ru/result/7708119944_ooo-sibirskaya-internet-kompaniya
t0 = time.time()
def find_info_comp_primer(url_one):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку
    """
    stroka = [] # собираем информацию сюда

    text_one = requests.get(url_one)
    text_write = text_one.text
    a1 = text_write.split('id="searchText" name="s" value=')[1]
    a2 = a1.split(' placeholder')[0] # name
    stroka.append(a2)

    a2 = a1.split('ИНН: ')[1].split(' ')[0] # INN
    stroka.append(a2)

    a2 = a1.split('Активы на ')[1].split('<b>')[1].split('</b>') # aktiv
    a2[0] # aktiv
    mln = a2[1].split(' (')[0] # млн.
    if 'млн.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(1)
    elif 'тыс.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(0.001)

    a2 = a1.split('Выручка за ')[1].split('<b>')[1].split(' </b>') # выручка
    mln = a2[1].split(' (')[0] # млн.
    if 'млн.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(1)
    elif 'тыс.' in mln:
        stroka.append(a2[0].replace(' ', ''))
        stroka.append(0.001)

    if 'численности ' in text_write:
        stroka.append(a1.split('численности ')[1].split('b>')[1].split('<')[0]) # численность человек
    else:
        stroka.append(0) 

    if 'Дополнительные данные ФНС' in text_write:
        a2 = a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[1].split('<')[0].rstrip().replace(' ', '')
        stroka.append(a2) # доходы 
        a2 = a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[2].split('<')[0].rstrip().replace(' ', '')
        stroka.append(a2) # расходы
    else:
        stroka.append(0)
        stroka.append(0)

    if "Уплачено налогов" in text_write:
        stroka.append(text_write.split("Уплачено налогов")[1].split('b>')[1].split('<')[0].rstrip()) # налоги уплаченные
    else:
        stroka.append(0)
    
    return stroka

url_primer = 'https://www.testfirm.ru/result/7702231594_ooo-direkt-star'

find_info_comp_primer(url_primer)
print(time.time() - t0)
#####
url_one = url_primer
stroka = [] # собираем информацию сюда
text_one = requests.get(url_one)
text_write = text_one.text
a1 = text_write.split('id="searchText" name="s" value=')[1]
a2 = a1.split(' placeholder')[0] # name
a2 = a1.split('ИНН: ')[1].split(' ')[0] # INN
a2 = a1.split('Активы на ')[1].split('<b>')[1].split('</b>') # aktiv
a2 = a2[1].split('(')[0] # млн.руб.
a2 = a1.split('Выручка за ')[1].split('<b>')[1].split(' </b>') # выручка
a2 = a2[1].split(' (')[0] # млн.руб.
a2 = a1.split('числен')[1]
print(a2) # показать кусок текста
print(a1) #  показать текст весь
##### проверить 57 место
#time.time() - t0 # 0.956343412399292
# name, inn, активы, выручка, персонал, доходы, расходы, налоги

# Анализ данных

In [ ]:
data = pd.DataFrame(Table_find_values)
data.columns = ['название', 'ИНН', 'активы','e' , 'выручка', 'e', 'персонал', 'доходы', 'расходы', 'налоги'] 
# name, inn, aktiv/активы, sale/выручка, персонал, доходы, расходы, налоги
data.tail()
#data['название'].value_counts()

,название,ИНН,активы,e,выручка,e,персонал,доходы,расходы,налоги
555,"""ТЕХНОЛОГИИ XXI ВЕК""",7743808295,10,1.0,"62,8",1,44,"62,8","54,6","4,51"
556,"""ИТ-ПРОФ""",7743869121,"50,2",1.0,"62,8",1,28,"90,3","80,2","2,98"
557,"""ТЕХНОЛОГИИ XXI ВЕК""",7743808295,10,1.0,"62,8",1,44,"62,8","54,6","4,51"
558,"""ИТ-ПРОФ""",7743869121,"50,2",1.0,"62,8",1,28,"90,3","80,2","2,98"
559,"""ТЕХНОЛОГИИ XXI ВЕК""",7743808295,10,1.0,"62,8",1,44,"62,8","54,6","4,51"


In [ ]:
data[540:560]
# 550 551 552 
# данные с 551 начали дублироваться 
#

,название,ИНН,активы,e,выручка,e,персонал,доходы,расходы,налоги
540,"""Современные бизнес технологии и системы""",6317084508,"38,3",1.0,"64,6",1,15,"64,8","32,4","2,53"
541,"""СК Гарант-Столица""",7728243354,"14,8",1.0,"64,5",1,8,"65,4","50,7","5,36"
542,"""НТПЦ""",7801483426,"22,4",1.0,"64,4",1,51,"64,6","51,3","5,88"
543,"""БИЗНЕС ИНФОРМ""",9705008020,"11,1",1.0,"64,4",1,31,"64,5","51,6","4,08"
544,"""Самарская мониторинговая компания Блокпост""",6316127043,"42,7",1.0,"64,3",1,16,"64,6","66,8",1
545,"""ЭЛВИНГ""",6670413195,16,1.0,"64,3",1,35,"65,1","45,9","4,03"
546,"""МираЛоджик СМ""",6670385082,"42,8",1.0,"64,2",1,48,"64,7","52,7","7,49"
547,"""СИДИКОМ НАВИГАЦИЯ""",7731593013,"92,1",1.0,"63,8",1,20,90,109,"3,28"
548,"""КЬЮГО""",7714958881,11,1.0,"63,7",1,22,"63,8","55,2","2,3"
549,"""Межрегиональный расчетный центр""",3460011342,"27,2",1.0,"63,3",1,29,"63,3","39,4","4,23"


In [149]:
#загрузить файл на лок.машину
data.to_csv('data_find_values_2.csv')  # запишем в файл
#files.download('data_find_values.csv') # сохраним в файл

In [ ]:
# загрузим файл на колаб
uploaded = files.upload()

In [ ]:
# загрузим сохраненные данные
data = pd.read_csv('data_find_values.csv', index_col='Unnamed: 0')
#data.columns = ['name', 'inn', 'активы', 'ед_изм_акт', 'выручка', 'ед_изм_выр', 'персонал', 'доходы', 'расходы', 'налоги'] 
data.head(3)

,название,ИНН,активы,e,выручка,e.1,персонал,доходы,расходы,налоги
0,"""Сибирская Интернет Компания""",7708119944,47403,1.0,52074,1,0,0,0,0
1,"""МЭЙЛ РУ""",7743001840,198742,1.0,45582,1,0,0,0,0
2,"""Центр программ лояльности""",7702770003,8723,1.0,32778,1,166,33174,30617,1 680


In [ ]:
#data['ед_изм_акт'].unique()
data['akt'] = 1 * data['ед_изм_акт'] / 1000000
#data['akt'] = data['akt'] * data['активы']
#data[data.query("ед_изм_акт	== 1000")] = 0.001
data.head(10)
data.query("ед_изм_акт	== 1000")

In [ ]:
print(data['ед_изм_акт'].value_counts())
print()
print(data['ед_изм_выр'].value_counts())

1000000    6498
1000          4
Name: ед_изм_акт, dtype: int64

1000000    6502
Name: ед_изм_выр, dtype: int64


In [ ]:
/content/data_find_values_2.csv